In [ ]:
import numpy as np
import os
import subprocess
import shutil
import time
from utils.XRD_Checker import *
import glob

path_root = ":: root directory"
path_cif = path_root+':: your own cif files'
path_run = path_root+"::fullprof work path"
path_data = path_root+"csv/ ::save path"
fullprof = "/fullprof/fp2k ::fp2k path"
path_error = path_root+"error:: error path"

cif_list = glob.glob(path_cif+"*.cif")

In [ ]:
def fullprof_f(gen_d, atm_d, i, f_dat):
    try:
        u= 0.004133
        v=-0.007618
        w=0.006255
        Shape1= 0.00000   #Eta
        Asy1= 0.04000  
        Asy2= 0.02000
        X = 0.018961
        Y = 0
        Thmin = 5
        Thmax = 105
        Step = 0.01
        num_data = int((Thmax-Thmin)/Step+1) 
        if i % 10000 == 0: print(i," at",time.strftime("%x %X",time.localtime(time.time())))
        fp = open(path_run+'base.pcr','r+')
        fpc = fp.readlines()
        fpc[25] = "         5.0000         0.0000          0.00\n"
        fpc[26] = "        15.0000         0.0000          0.00\n"
        fpc[27] = "        25.0000         0.0000          0.00\n"
        fpc[28] = "        35.0000         0.0000          0.00\n"
        fpc[29] = "        45.0000         0.0000          0.00\n"
        fpc[30] = "        55.0000         0.0000          0.00\n"
        fpc[31] = "        65.0000         0.0000          0.00\n"
        fpc[32] = "        75.0000         0.0000          0.00\n"
        fpc[33] = "        85.0000         0.0000          0.00\n"
        fpc[34] = "        95.0000         0.0000          0.00\n"
        fpc[35] = "       105.0000         0.0000          0.00\n"

        fpc[52] = " " + str(gen_d[i,16]) + "  0   0   0   0   0   0       100.0000   0   0 \n"
        fpc[61] = gen_d[i,7] + "                <--Space group symbol\n"
        while True:
            del fpc[63]
            if "!-------> Profile Parameters for Pattern" in fpc[63]:
                break
        for j in range(gen_d[i,16]):
            if atm_d[i,j,7] == 0.0:
                fpc.insert(63+j*2, atm_d[i,j,0]+"    "+atm_d[i,j,1][:-2]+"     "+"{:8.5f}".format(atm_d[i,j,4])+"  "\
                        +"{:8.5f}".format(atm_d[i,j,5])+"  "+"{:8.5f}".format(atm_d[i,j,6])+"  "+"{:8.5f}".format(1.0)\
                        +"   "+"{:8.5f}".format(atm_d[i,j,8]*atm_d[i,j,2]/sg_m[gen_d[i,6]])+"   0   0   0   0 \n")
            else:
                fpc.insert(63+j*2, atm_d[i,j,0]+"    "+atm_d[i,j,1][:-2]+"     "+"{:8.5f}".format(atm_d[i,j,4])+"  "\
                        +"{:8.5f}".format(atm_d[i,j,5])+"  "+"{:8.5f}".format(atm_d[i,j,6])+"  "+"{:8.5f}".format(atm_d[i,j,7])\
                        +"   "+"{:8.5f}".format(atm_d[i,j,8]*atm_d[i,j,2]/sg_m[gen_d[i,6]])+"   0   0   0   0 \n")         
            fpc.insert(63+j*2+1, "                  0.00     0.00     0.00     0.00      0.00 \n")
        for j in range(len(fpc)):

            if "!Ipr Ppl Ioc Ls1 Ls2 Ls3 Prf Ins Hkl Fou Ana"in fpc[j]:
                fpc[j+1] = "   0   0   0   0   4   0   3  10   0   0   0  !-> Patt#: 1\n"
            elif "!     Thmin       Step       Thmax    PSD    Sent0" in fpc[j]:
                fpc[j+1] = "     %0.4f   %0.6f   %0.4f   0.000   0.000 \n" %(Thmin,Step,Thmax)        
            elif "!Irf Npr Jtyp  Nsp_Ref Ph_Shift for Pattern" in fpc[j]:
                fpc[j+1] = "   0   5    2      0      0 \n"
            elif "! Pr1    Pr2    Pr3   Brind.   Rmua   Rmub   Rmuc     for Pattern"in fpc[j]:
                fpc[j+1] = "  -1.000  -1.000  -1.000  1.000  0.000  0.000  0.000\n"
        for j in range(len(fpc)):
            if "!Job Npr Nba Nex Nsc Nor Iwg Ilo Res Ste Uni Cor Anm Int"in fpc[j]:
                fpc[j+1] = "   2   7  11   2   0   1   0   0   0   0   0   0   0   0  !-> Patt#: 1\n"    ###Job 2: X-ray , 3: Neutron
            elif "! Lambda1  Lambda2    Ratio    Bkpos    Wdt    Cthm     muR   AsyLim   Rpolarz  2nd-muR -> Patt# 1"in fpc[j]:  #### Cthm 0.7998: X-ray, 0: Neutron
                fpc[j+1] = ' 1.540560 1.544390  0.50000   40.000 50.0000  0.7998  0.0000   40.00    0.0000  0.0000 \n'
            elif "!  Scale        Shape1      Bov      Str1      Str2      Str3   Strain-Model" in fpc[j]:
                fpc[j+1] = " 0.10000E-05"+"   "+str(Shape1)+"   0.00000   0.00000   0.00000   0.00000       0 \n"
            elif "!       U         V          W           X          Y        GauSiz   LorSiz Size-Model" in fpc[j]:
                fpc[j+1] = "   "+str(u)+"  "+str(v)+"   "+str(w)+"   "+str(X)+"   "+str(Y)+"   0.000000   0.000000    0\n"
            elif "!     a          b         c        alpha      beta       gamma" in fpc[j]:
                fpc[j+1] = "   "+"{:8.6f}".format(gen_d[i,8])+"  "+"{:8.6f}".format(gen_d[i,9])+"  "+"{:8.6f}".format(gen_d[i,10])\
                           +"  "+"{:8.6f}".format(gen_d[i,11])+"  "+"{:8.6f}".format(gen_d[i,12])+"  "+"{:8.6f}".format(gen_d[i,13])+"\n"
            elif "!  Pref1    Pref2      Asy1     Asy2     Asy3     Asy4  " in fpc[j]:
                fpc[j+1] ="  0.00000  0.00000 "+str(Asy1)+"  "+str(Asy2)+"  "+"0.00000  0.00000\n"            
                break
        fp.seek(0)
        f = open(path_run+'%d.pcr' %i, 'w')
        for j in range(len(fpc)):
            f.write(fpc[j])
        fp.close()
        f.close()
        proc = subprocess.check_call([fullprof,' %d.pcr' %i], stdout=subprocess.DEVNULL)
        fs = open(path_run+'%d.sim'%i,'r')
        line = fs.readline()
        for j in range(math.floor(len(data)/8)):
            line = fs.readline().split()
            for k in range(8):
                data[j*8+k] = line[k]
        fs.close()
        fs = open(path_run+'%d.sim'%i,'r')  
        line = fs.readline()        
        for q in range(math.floor(len(data)/8)*8,len(data)):
            data[q] = fs.readline().split()[0]
        fs.close()
        mi = np.max(data)
        fp = open(path_run+'%d.pcr' %i,'r+')
        fpc = fp.readlines()
        for j in range(gen_d[i,16]):
            if atm_d[i,j,7] == 0.0:
                fpc[63+j*2] = atm_d[i,j,0]+"    "+atm_d[i,j,1][:-2]+"     "+"{:8.5f}".format(atm_d[i,j,4])+"  "\
                        +"{:8.5f}".format(atm_d[i,j,5])+"  "+"{:8.5f}".format(atm_d[i,j,6])+"  "+"{:8.5f}".format(1.0)\
                        +"   "+"{:8.5f}".format(atm_d[i,j,8]*atm_d[i,j,2]/sg_m[gen_d[i,6]])+"   0   0   0   0 \n"
            else:
                fpc[63+j*2] = atm_d[i,j,0]+"    "+atm_d[i,j,1][:-2]+"     "+"{:8.5f}".format(atm_d[i,j,4])+"  "\
                        +"{:8.5f}".format(atm_d[i,j,5])+"  "+"{:8.5f}".format(atm_d[i,j,6])+"  "+"{:8.5f}".format(atm_d[i,j,7])\
                        +"   "+"{:8.5f}".format(atm_d[i,j,8]*atm_d[i,j,2]/sg_m[gen_d[i,6]])+"   0   0   0   0 \n"            
        for j in range(len(fpc)):
            if "!Job Npr Nba Nex Nsc Nor Iwg Ilo Res Ste Uni Cor Anm Int"in fpc[j]:
                fpc[j+1] = "   2   7  11   2   0   1   0   0   0   0   0   0   0   0  !-> Patt#: 1\n"    ###Job 2: X-ray , 3: Neutron
            elif "! Lambda1  Lambda2    Ratio    Bkpos    Wdt    Cthm     muR   AsyLim   Rpolarz  2nd-muR -> Patt# 1"in fpc[j]:  #### Cthm 0.7998: X-ray, 0: Neutron
                fpc[j+1] = ' 1.540560 1.544390  0.50000   40.000 50.0000  0.7998  0.0000   40.00    0.0000  0.0000 \n'

            elif "!  Scale        Shape1      Bov      Str1      Str2      Str3   Strain-Model" in fpc[j]:
                fpc[j+1] = "{:13.5e}".format(float(fpc[j+1].split()[0])/(mi+1)*9900)+"   "+str(Shape1)+"  0.00000  0.00000  0.00000  0.00000   0 \n"
                break   
        fp.seek(0)
        for j in range(len(fpc)):
            fp.write(fpc[j])
        fp.close()
        proc = subprocess.check_call([fullprof,' %d.pcr' %i], stdout=subprocess.DEVNULL)   
        fs = open(path_run+'%d.sim'%i,'r')
        line = fs.readline()
        for j in range(math.floor(len(data)/8)):
            line = fs.readline().split()
            for k in range(8):
                data[j*8+k] = line[k]
        fs.close()
        fs = open(path_run+'%d.sim'%i,'r')
        line = fs.readline()        

        for q in range(math.floor(len(data)/8)*8,len(data)):
            data[q] = fs.readline().split()[0]
        fs.close()

        mi = np.max(data)

        if not(7000 < mi < 14000):
            fp = open(path_run+'%d.pcr' %i,'r+')
            fpc = fp.readlines()

            for j in range(gen_d[i,16]):
                if atm_d[i,j,7] == 0.0:
                    fpc[63+j*2] = atm_d[i,j,0]+"    "+atm_d[i,j,1][:-2]+"     "+"{:8.5f}".format(atm_d[i,j,4])+"  "\
                            +"{:8.5f}".format(atm_d[i,j,5])+"  "+"{:8.5f}".format(atm_d[i,j,6])+"  "+"{:8.5f}".format(1.0)\
                            +"   "+"{:8.5f}".format(atm_d[i,j,8]*atm_d[i,j,2]/sg_m[gen_d[i,6]])+"   0   0   0   0 \n"
                else:
                    fpc[63+j*2] = atm_d[i,j,0]+"    "+atm_d[i,j,1][:-2]+"     "+"{:8.5f}".format(atm_d[i,j,4])+"  "\
                            +"{:8.5f}".format(atm_d[i,j,5])+"  "+"{:8.5f}".format(atm_d[i,j,6])+"  "+"{:8.5f}".format(atm_d[i,j,7])\
                            +"   "+"{:8.5f}".format(atm_d[i,j,8]*atm_d[i,j,2]/sg_m[gen_d[i,6]])+"   0   0   0   0 \n"            

            for j in range(len(fpc)):
                if "!Job Npr Nba Nex Nsc Nor Iwg Ilo Res Ste Uni Cor Anm Int"in fpc[j]:
                    fpc[j+1] = "   2   7  11   2   0   1   0   0   0   0   0   0   0   0  !-> Patt#: 1\n"    ###Job 2: X-ray , 3: Neutron
                elif "! Lambda1  Lambda2    Ratio    Bkpos    Wdt    Cthm     muR   AsyLim   Rpolarz  2nd-muR -> Patt# 1"in fpc[j]:  #### Cthm 0.7998: X-ray, 0: Neutron
                    fpc[j+1] = ' 1.540560 1.544390  0.50000   40.000 50.0000  0.7998  0.0000   40.00    0.0000  0.0000 \n'              
                elif "!  Scale        Shape1      Bov      Str1      Str2      Str3   Strain-Model" in fpc[j]:  
                    fpc[j+1] = "{:13.5e}".format(float(fpc[j+1].split()[0])/(mi+1)*9900)+"   "+str(Shape1)+"  0.00000  0.00000  0.00000  0.00000   0 \n"
                    break   

            fp.seek(0)
            for j in range(len(fpc)):
                fp.write(fpc[j])
            fp.close()
            proc = subprocess.check_call([fullprof,' %d.pcr' %i], stdout=subprocess.DEVNULL)   
            fs = open(path_run+'%d.sim'%i,'r')
            line = fs.readline()
            for j in range(math.floor(len(data)/8)):
                line = fs.readline().split()
                for k in range(8):
                    data[j*8+k] = line[k]
            fs.close()
            fs = open(path_run+'%d.sim'%i,'r')
            line = fs.readline()
            for q in range(math.floor(len(data)/8)*8,len(data)):
                data[q] = fs.readline().split()[0]
            fs.close()
            mi = np.max(data)
        fp = open(path_run+'%d.pcr' %i,'r+')
        fpc = fp.readlines()
        fpc[25] = "         5.0000         100.0000          0.00\n"
        fpc[26] = "        15.0000         100.0000          0.00\n"
        fpc[27] = "        25.0000         100.0000          0.00\n"
        fpc[28] = "        35.0000         100.0000          0.00\n"
        fpc[29] = "        45.0000         100.0000          0.00\n"
        fpc[30] = "        55.0000         100.0000          0.00\n"
        fpc[31] = "        65.0000         100.0000          0.00\n"
        fpc[32] = "        75.0000         100.0000          0.00\n"
        fpc[33] = "        85.0000         100.0000          0.00\n"
        fpc[34] = "        95.0000         100.0000          0.00\n"
        fpc[35] = "       105.0000         100.0000          0.00\n"
        for j in range(gen_d[i,16]):
            if atm_d[i,j,7] == 0.0:
                fpc[63+j*2] = atm_d[i,j,0]+"    "+atm_d[i,j,1][:-2]+"     "+"{:8.5f}".format(atm_d[i,j,4])+"  "\
                        +"{:8.5f}".format(atm_d[i,j,5])+"  "+"{:8.5f}".format(atm_d[i,j,6])+"  "+"{:8.5f}".format(1.0)\
                        +"   "+"{:8.5f}".format(atm_d[i,j,8]*atm_d[i,j,2]/sg_m[gen_d[i,6]])+"   0   0   0   0 \n"
            else:
                fpc[63+j*2] = atm_d[i,j,0]+"    "+atm_d[i,j,1][:-2]+"     "+"{:8.5f}".format(atm_d[i,j,4])+"  "\
                        +"{:8.5f}".format(atm_d[i,j,5])+"  "+"{:8.5f}".format(atm_d[i,j,6])+"  "+"{:8.5f}".format(atm_d[i,j,7])\
                        +"   "+"{:8.5f}".format(atm_d[i,j,8]*atm_d[i,j,2]/sg_m[gen_d[i,6]])+"   0   0   0   0 \n"           
        for j in range(len(fpc)):
            if "!Job Npr Nba Nex Nsc Nor Iwg Ilo Res Ste Uni Cor Anm Int"in fpc[j]:
                fpc[j+1] = "   2   7  11   2   0   1   0   0   0   0   0   0   0   0  !-> Patt#: 1\n"    ###Job 2: X-ray , 3: Neutron
            elif "! Lambda1  Lambda2    Ratio    Bkpos    Wdt    Cthm     muR   AsyLim   Rpolarz  2nd-muR -> Patt# 1"in fpc[j]:  #### Cthm 0.7998: X-ray, 0: Neutron
                fpc[j+1] = ' 1.540560 1.544390  0.50000   40.000 50.0000  0.7998  0.0000   40.00    0.0000  0.0000 \n'            
            elif "!  Scale        Shape1      Bov      Str1      Str2      Str3   Strain-Model" in fpc[j]:
                fpc[j+1] = "{:13.5e}".format(float(fpc[j+1].split()[0])/mi*9900)+"   "+str(Shape1)+"   0.00000   0.00000   0.00000   0.00000       0 \n"
                break
        fp.seek(0)
        for j in range(len(fpc)):
            fp.write(fpc[j])
        fp.close()
        proc = subprocess.check_call([fullprof,' %d.pcr' %i], stdout=subprocess.DEVNULL)    
        fs = open(path_run+'%d.sim'%i,'r')
        line = fs.readline()
        for j in range(math.floor(len(data)/8)):
            line = fs.readline().split()
            for k in range(8):
                data[j*8+k] = line[k]
        fs.close()
        fs = open(path_run+'%d.sim'%i,'r')
        line = fs.readline()        
        for q in range(math.floor(len(data)/8)*8,len(data)):
            data[q] = fs.readline().split()[0]
        fs.close()
        mi = np.max(data)
        fd = open(path_data+f_dat[i].split(".")[0]+".csv",'w')
        for j in range(len(data)):
            fd.write("{:1d}".format(int(data[j]/mi*10000))+"\n")
        fd.write("{:8.6f}".format(gen_d[i,8])+"\n")        # A
        fd.write("{:8.6f}".format(gen_d[i,9])+"\n")        # B
        fd.write("{:8.6f}".format(gen_d[i,10])+"\n")       # C   
        fd.write("{:8.6f}".format(gen_d[i,11])+"\n")       # alpha
        fd.write("{:8.6f}".format(gen_d[i,12])+"\n")       # beta
        fd.write("{:8.6f}".format(gen_d[i,13])+"\n")       # gamma 
        fd.write("{:8.6f}".format(gen_d[i,14])+"\n")       #Volume
        fd.write("{:8.6f}".format(u)+"\n")                 # u
        fd.write("{:8.6f}".format(v)+"\n")                 # v
        fd.write("{:8.6f}".format(w)+"\n")                 # w
        fd.write("{:8.6f}".format(X)+"\n")                 # X
        fd.write("{:8.6f}".format(Y)+"\n")                 # Y
        fd.write("{:8.5f}".format(Shape1)+"\n")            # eta
        fd.write(str(gen_d[i,17])+'\n')                    # crystal system
        fd.write(str(gen_d[i,18])+'\n')                    # extinction group
        fd.write(str(gen_d[i,6])+'\n')                     # space group

        fd.close()
        os.remove(path_run+'%d.sim'%i)
    except:
        shutil.move(path_cif+f_dat[i].split(".")[0]+".cif",error_path)

In [ ]:
Thmin = 5
Thmax = 105
Step = 0.08
num_data = int((Thmax-Thmin)/Step+1) 

In [ ]:
# Simulated Pattern Generation
import random
import math

start = time.time()

os.chdir(path_run)

parameter_list = []
sychrotron = 0
noise =0
data = np.zeros(num_data)

In [ ]:
from joblib import Parallel, delayed
import math
from tqdm.auto import tqdm

path_data = path_root+"/"
batch = 64
os.mkdir(path_data)


gen_d, atm_d, n_data, f_dat =  DataExtraction(path_cif,batch)
gen_d, atm_d, n_data =  SpaceGroupConversion(gen_d, atm_d, n_data)
gen_d, atm_d, n_data, f_dat =  GeneralInfromCheck(gen_d, atm_d, n_data, f_dat)
gen_d, atm_d, n_data, f_dat = AtomicInformCheck(gen_d, atm_d, n_data, f_dat)
gen_d, atm_d, n_data, f_dat = SpaceGroupCheck1(gen_d, atm_d, n_data, f_dat)
gen_d, atm_d, n_data, f_dat = Rejection(gen_d, atm_d, n_data, f_dat)

[fullprof_f(gen_d, atm_d, i, f_dat) for i in tqdm(range(n_data))]